In [ ]:
!pip install requirements.txt
from sklearn.feature_extraction.text import TfidfVectorizer
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import urllib.parse
import matplotlib.pyplot as plt

In [ ]:
## Loading data

In [ ]:
def loadFile(name):
    filepath = os.path.join(str(os.getcwd()), name)
    with open(filepath,'r') as f:
        data = f.readlines()
    return [str(urllib.parse.unquote(e)) for e in list(set(data))]

In [ ]:
badQueries = loadFile('badqueries.txt')

badCount = len(badQueries)

In [ ]:
validQueries = loadFile('goodqueries.txt')

validCount = len(validQueries)

In [ ]:
queries = badQueries + validQueries

In [ ]:
print("bad: ", badCount)
print("good: ", validCount)
print("all: ", badCount + validCount)

In [ ]:
yBad = [1 for i in range(0, len(badQueries))]  #labels, 1 for malicious and 0 for clean
yGood = [0 for i in range(0, len(validQueries))]

In [ ]:
y = yBad + yGood

In [ ]:
## Preparing the dataset

In [ ]:
vectorizer = TfidfVectorizer(min_df = 0.0, analyzer="char", sublinear_tf=True, ngram_range=(1,3)) #converting data to vectors
X = vectorizer.fit_transform(queries)

In [ ]:
X

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #splitting data

In [ ]:
## Training

In [ ]:
lgs = LogisticRegression(class_weight={1: 2 * validCount / badCount, 0: 1.0}) # class_weight='balanced')
lgs.fit(X_train, y_train) #training our model

In [ ]:
## Metrics

In [ ]:
predicted = lgs.predict(X_test)

In [ ]:
fpr, tpr, _ = metrics.roc_curve(y_test, (lgs.predict_proba(X_test)[:, 1]))
auc = metrics.auc(fpr, tpr)

In [ ]:
print("Bad samples: %d" % badCount)
print("Good samples: %d" % validCount)
print("Baseline Constant negative: %.6f" % (validCount / (validCount + badCount)))
print("------------")
print("Accuracy: %f" % lgs.score(X_test, y_test))  #checking the accuracy
print("Precision: %f" % metrics.precision_score(y_test, predicted))
print("Recall: %f" % metrics.recall_score(y_test, predicted))
print("F1-Score: %f" % metrics.f1_score(y_test, predicted))
print("AUC: %f" % auc)

In [ ]:
## Testing

In [ ]:
url = "index.php?q=../../../../../../../../../etc/passwd"
X_url = vectorizer.transform([url])
url_predicted = lgs.predict(X_url)
print("Is bad ?", bool(url_predicted[0]))

In [ ]:
url = "index.php?q=toto"
X_url = vectorizer.transform([url])
url_predicted = lgs.predict(X_url)
print("Is bad ?", bool(url_predicted[0]))